In [ ]:
#| hide
from auditus.core import *

# auditus

> Simple Audio Embeddings

Python module to simple access state-of-the-art audio embeddings. Like [SentenceTransformers](https://sbert.net/) for audio.

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install auditus in Development mode

```sh
# make sure auditus package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to auditus
$ nbdev_prepare
```

## Usage

### Installation

```sh
$ pip install auditus
```


[repo]: https://github.com/CarloLepelaars/auditus
[docs]: https://CarloLepelaars.github.io/auditus/
[pypi]: https://pypi.org/project/auditus/

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/CarloLepelaars/auditus
[docs]: https://CarloLepelaars.github.io/auditus/
[pypi]: https://pypi.org/project/auditus/
[conda]: https://anaconda.org/CarloLepelaars/auditus

## How to use

## Simple embeddings

In [ ]:
# from auditus import AudioTransformer, load_audio

# model = AudioTransformer("MIT/ast-finetuned-audioset-10-10-0.4593")

In [ ]:
# audio = load_audio("../test_files/XC119042.ogg")
# embeddings = model.encode(audio)
# embeddings.shape

# TODO Similarities

## Type Dispatch

Filenames or NumPy arrays. Automatically dispatched with fasttransform

## Similarities and Search

## Audio Representations

## Quantization